In [1]:
import requests
from wrapped_auth import WhoopAPI

# Create an instance of WhoopAPI
client_id = '86693dab-f7c4-448e-8fce-af6b4e5b929d'
client_secret = '0440cc1f0a10a2ef9eefdeeb7f2c1deaa489ec57aa9494a38c1826252bddda7f'
redirect_uri = 'http://localhost:8000/redirect.html'
all_scopes = ["read:profile", "read:recovery", "read:workout", "read:sleep", "read:body_measurement", "read:cycles"]

whoop_api = WhoopAPI(client_id, client_secret, redirect_uri, all_scopes)

# Step 1: Get the authorization URL and prompt user to open it in a browser
authorization_url = whoop_api.get_authorization_url()
print(f"Please go to this URL and authorize the app: {authorization_url}")

# Step 2: Get the authorization response from the user
authorization_response = input('Enter the full callback URL: ')

# Step 3: Get the access token using the authorization response
whoop_api.get_access_token(authorization_response)

# Set up headers for authorization
headers = {
    'Authorization': f'Bearer {whoop_api.access_token}'
}

Please go to this URL and authorize the app: https://api.prod.whoop.com/oauth/oauth2/auth?response_type=code&client_id=86693dab-f7c4-448e-8fce-af6b4e5b929d&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Fredirect.html&scope=read%3Aprofile+read%3Arecovery+read%3Aworkout+read%3Asleep+read%3Abody_measurement+read%3Acycles&state=yOlMyNjOzOYw9iTblbGgb5VXolG7Ez


Enter the full callback URL:  http://localhost:8000/redirect.html?code=9c6L2Gct_X3EV-xEoOZqnWpF0Z0X3P-ZKInSb-By5jQ.pTPgEuVZlufhLcq1FTKZJSLTnIg5BXfAkNZ7xXoAXyo&scope=read%3Aprofile%20read%3Arecovery%20read%3Aworkout%20read%3Asleep%20read%3Abody_measurement%20read%3Acycles&state=yOlMyNjOzOYw9iTblbGgb5VXolG7Ez


In [2]:
import pandas as pd
from pandas import json_normalize

In [3]:
import langchain
import openai 
import json 

import os
from dotenv import load_dotenv

# Provide the path to the .env file
env_path = '/Users/galampley/Documents/api_keys.env'  # Replace this with the actual path to your .env file

# Load environment variables from .env file
load_dotenv(dotenv_path=env_path)

# Access environment variables
openai.api_key = os.getenv('OPENAI_API_KEY')

# flatten most recent 10 records into pd DF

In [27]:
def get_sleep_flat_v1(id=None, user_id=None, created_at=None, updated_at=None, start=None, end=None, timezone_offset=None, nap=None, score_state=None, score_respiratory_rate=None,
                    score_sleep_performance_percentage=None,
                    score_sleep_consistency_percentage=None,
                    score_sleep_efficiency_percentage=None,
                    score_stage_summary_total_in_bed_time_milli=None,
                    score_stage_summary_total_awake_time_milli=None,
                    score_stage_summary_total_no_data_time_milli=None,
                    score_stage_summary_total_light_sleep_time_milli=None,
                    score_stage_summary_total_slow_wave_sleep_time_milli=None,
                    score_stage_summary_total_rem_sleep_time_milli=None,
                    score_stage_summary_sleep_cycle_count=None,
                    score_stage_summary_disturbance_count=None,
                    score_sleep_needed_baseline_milli=None,
                    score_sleep_needed_need_from_sleep_debt_milli=None,
                    score_sleep_needed_need_from_recent_strain_milli=None,
                    score_sleep_needed_need_from_recent_nap_milli=None):
    
    sleep_endpoint_url = 'https://api.prod.whoop.com/developer/v1/activity/sleep'
    response = requests.get(sleep_endpoint_url, headers=headers)
    response_json = response.json()
    
    # Flatten the main records
    df_main = json_normalize(response_json['records'], sep='_')
    
    # Create empty DataFrames for score, stage_summary, and sleep_needed
    df_score = pd.DataFrame()
    df_stage_summary = pd.DataFrame()
    df_sleep_needed = pd.DataFrame()
    
    # Loop through each record to flatten nested fields
    for record in response_json['records']:
        temp_df_score = json_normalize(record['score'], sep='_')
        temp_df_score['id'] = record['id']
        
        temp_df_stage_summary = json_normalize(record['score']['stage_summary'], sep='_')
        temp_df_stage_summary['id'] = record['id']
        
        temp_df_sleep_needed = json_normalize(record['score']['sleep_needed'], sep='_')
        temp_df_sleep_needed['id'] = record['id']
        
        df_score = pd.concat([df_score, temp_df_score])
        df_stage_summary = pd.concat([df_stage_summary, temp_df_stage_summary])
        df_sleep_needed = pd.concat([df_sleep_needed, temp_df_sleep_needed])
    
    # Merge all the DataFrames based on the 'id' field
    df = pd.merge(df_main, df_score, on='id')
    df = pd.merge(df, df_stage_summary, on='id')
    df = pd.merge(df, df_sleep_needed, on='id')

    # Drop duplicate columns
    duplicate_columns = ['respiratory_rate', 'sleep_performance_percentage', 'sleep_consistency_percentage',
                         'sleep_efficiency_percentage', 'stage_summary_total_in_bed_time_milli',
                         'stage_summary_total_awake_time_milli', 'stage_summary_total_no_data_time_milli',
                         'stage_summary_total_light_sleep_time_milli', 'stage_summary_total_slow_wave_sleep_time_milli',
                         'stage_summary_total_rem_sleep_time_milli', 'stage_summary_sleep_cycle_count',
                         'stage_summary_disturbance_count', 'sleep_needed_baseline_milli',
                         'sleep_needed_need_from_sleep_debt_milli', 'sleep_needed_need_from_recent_strain_milli',
                         'sleep_needed_need_from_recent_nap_milli', 'total_in_bed_time_milli',
                         'total_awake_time_milli', 'total_no_data_time_milli',
                         'total_light_sleep_time_milli', 'total_slow_wave_sleep_time_milli',
                         'total_rem_sleep_time_milli', 'sleep_cycle_count', 'disturbance_count',
                         'baseline_milli', 'need_from_sleep_debt_milli',
                         'need_from_recent_strain_milli', 'need_from_recent_nap_milli']

    df.drop(columns=duplicate_columns, inplace=True)
    
    return df

# Display the DataFrame
get_sleep_flat_v1()

,id,user_id,created_at,updated_at,start,end,timezone_offset,nap,score_state,score_stage_summary_total_in_bed_time_milli,...,score_stage_summary_sleep_cycle_count,score_stage_summary_disturbance_count,score_sleep_needed_baseline_milli,score_sleep_needed_need_from_sleep_debt_milli,score_sleep_needed_need_from_recent_strain_milli,score_sleep_needed_need_from_recent_nap_milli,score_respiratory_rate,score_sleep_performance_percentage,score_sleep_consistency_percentage,score_sleep_efficiency_percentage
0,781257341,7036784,2023-09-22T10:39:00.978Z,2023-09-22T13:47:19.278Z,2023-09-22T05:53:24.852Z,2023-09-22T13:29:40.090Z,-07:00,False,SCORED,27374277,...,6,22,27887569,4940512,1460085,0,17.578125,66.0,89.0,85.361190
1,780520172,7036784,2023-09-21T13:40:26.533Z,2023-09-21T13:40:31.454Z,2023-09-21T05:45:02.798Z,2023-09-21T13:29:50.548Z,-07:00,False,SCORED,27886788,...,7,13,27887867,5993730,281277,0,17.402344,71.0,79.0,87.525530
2,779342236,7036784,2023-09-20T11:21:19.182Z,2023-09-20T13:40:40.853Z,2023-09-20T05:52:17.244Z,2023-09-20T13:23:08.419Z,-07:00,False,SCORED,27050214,...,5,19,27888166,3422269,2507362,0,17.402344,65.0,66.0,80.703010
3,778465925,7036784,2023-09-19T13:05:21.938Z,2023-09-19T13:40:38.089Z,2023-09-19T04:36:56.910Z,2023-09-19T13:30:13.000Z,-07:00,False,SCORED,31996090,...,8,16,27888465,3738803,1410973,0,17.402344,84.0,54.0,86.619064
4,777277162,7036784,2023-09-18T10:46:29.871Z,2023-09-18T13:45:55.665Z,2023-09-18T04:12:33.549Z,2023-09-18T13:29:38.480Z,-07:00,False,SCORED,32847188,...,5,13,27888763,4652596,988314,0,18.017578,83.0,46.0,85.650170
5,776718852,7036784,2023-09-17T17:11:16.560Z,2023-09-17T17:11:21.844Z,2023-09-17T07:37:43.864Z,2023-09-17T17:06:49.612Z,-07:00,False,SCORED,33971751,...,1,11,27889062,5297342,1851072,0,17.929688,79.0,63.0,82.181840
6,775603575,7036784,2023-09-16T13:18:57.487Z,2023-09-16T14:43:03.228Z,2023-09-16T06:18:28.132Z,2023-09-16T14:29:16.868Z,-07:00,False,SCORED,29448736,...,4,9,27889361,3837054,2470203,0,17.695312,76.0,45.0,88.574356
7,774734315,7036784,2023-09-15T13:35:52.980Z,2023-09-15T17:21:50.004Z,2023-09-15T08:30:01.908Z,2023-09-15T17:07:13.150Z,-10:00,False,SCORED,31031242,...,8,16,27889660,7668000,403182,-4296117,17.578125,81.0,54.0,83.819470
8,774157050,7036784,2023-09-14T23:53:26.784Z,2023-09-14T23:53:31.303Z,2023-09-14T20:09:38.078Z,2023-09-14T21:48:39.004Z,-10:00,True,SCORED,5449686,...,1,2,27889959,7668000,269021,0,19.365234,12.0,42.0,94.903410
9,773807465,7036784,2023-09-14T13:43:54.155Z,2023-09-14T13:43:58.602Z,2023-09-14T08:07:12.744Z,2023-09-14T13:03:46.636Z,-10:00,False,SCORED,17793892,...,2,12,27889959,5470997,2071361,0,17.636719,45.0,60.0,89.413680


# all records flattened, no function wrap for testing

In [14]:
sleep_endpoint_url = 'https://api.prod.whoop.com/developer/v1/activity/sleep'
headers = {
    'Authorization': f'Bearer {whoop_api.access_token}'
}
all_records = []
next_token = None

df_main = pd.DataFrame()
df_score = pd.DataFrame()
df_stage_summary = pd.DataFrame()
df_sleep_needed = pd.DataFrame()

while True:
    params = {}
    if next_token:
        params["nextToken"] = next_token

    response = requests.get(sleep_endpoint_url, headers=headers, params=params)
    all_data = response.json()

    all_records.extend(all_data.get("records", []))
    next_token = all_data.get("next_token")

    if not next_token:
        break

for record in all_records:
    temp_df_main = json_normalize(record, sep='_')
    df_main = pd.concat([df_main, temp_df_main])

    temp_df_score = json_normalize(record['score'], sep='_')
    temp_df_score['id'] = record['id']
    df_score = pd.concat([df_score, temp_df_score])

    temp_df_stage_summary = json_normalize(record['score']['stage_summary'], sep='_')
    temp_df_stage_summary['id'] = record['id']
    df_stage_summary = pd.concat([df_stage_summary, temp_df_stage_summary])

    temp_df_sleep_needed = json_normalize(record['score']['sleep_needed'], sep='_')
    temp_df_sleep_needed['id'] = record['id']
    df_sleep_needed = pd.concat([df_sleep_needed, temp_df_sleep_needed])

df_2 = pd.merge(df_main, df_score, on='id')
df_2 = pd.merge(df_2, df_stage_summary, on='id')
df_2 = pd.merge(df_2, df_sleep_needed, on='id')

# Drop duplicate columns
duplicate_columns = ['respiratory_rate',
       'sleep_performance_percentage', 'sleep_consistency_percentage',
       'sleep_efficiency_percentage', 'stage_summary_total_in_bed_time_milli',
       'stage_summary_total_awake_time_milli',
       'stage_summary_total_no_data_time_milli',
       'stage_summary_total_light_sleep_time_milli',
       'stage_summary_total_slow_wave_sleep_time_milli',
       'stage_summary_total_rem_sleep_time_milli',
       'stage_summary_sleep_cycle_count', 'stage_summary_disturbance_count',
       'sleep_needed_baseline_milli',
       'sleep_needed_need_from_sleep_debt_milli',
       'sleep_needed_need_from_recent_strain_milli',
       'sleep_needed_need_from_recent_nap_milli', 'total_in_bed_time_milli',
       'total_awake_time_milli', 'total_no_data_time_milli',
       'total_light_sleep_time_milli', 'total_slow_wave_sleep_time_milli',
       'total_rem_sleep_time_milli', 'sleep_cycle_count', 'disturbance_count','baseline_milli',
       'need_from_sleep_debt_milli', 'need_from_recent_strain_milli',
       'need_from_recent_nap_milli']
df_2.drop(columns=duplicate_columns, inplace=True)

print(df_2.columns)

Index(['id', 'user_id', 'created_at', 'updated_at', 'start', 'end',
       'timezone_offset', 'nap', 'score_state',
       'score_stage_summary_total_in_bed_time_milli',
       'score_stage_summary_total_awake_time_milli',
       'score_stage_summary_total_no_data_time_milli',
       'score_stage_summary_total_light_sleep_time_milli',
       'score_stage_summary_total_slow_wave_sleep_time_milli',
       'score_stage_summary_total_rem_sleep_time_milli',
       'score_stage_summary_sleep_cycle_count',
       'score_stage_summary_disturbance_count',
       'score_sleep_needed_baseline_milli',
       'score_sleep_needed_need_from_sleep_debt_milli',
       'score_sleep_needed_need_from_recent_strain_milli',
       'score_sleep_needed_need_from_recent_nap_milli',
       'score_respiratory_rate', 'score_sleep_performance_percentage',
       'score_sleep_consistency_percentage',
       'score_sleep_efficiency_percentage'],
      dtype='object')


In [8]:
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI


In [9]:
llm = ChatOpenAI(    
    openai_api_key=openai.api_key , 
    model_name="gpt-3.5-turbo", 
    temperature=0.0
    )

In [10]:
python = PythonAstREPLTool(locals={"df": df_2}) # set access of python_repl tool to the dataframe

In [11]:
df_columns = df_2.columns.to_list() # print column names of df

In [65]:
from datetime import datetime
def today_date(query:str) -> str:
    return datetime.now().strftime("%Y-%m-%d")

In [91]:
tools = [
    Tool(
        name = "Whoop Sleep Data",
        func=python.run,
        description = f"""
        Useful for when you need to answer questions about Whoop data stored in pandas dataframe 'df'. 
        Run python pandas operations on 'df' to help you get the right answer.
        'df' has the following columns: {df_columns}.
        
        Each column ({df_columns}) from 'df' is defined below between '####'
        
        ####
        id: Unique identifier for the sleep activity.
        user_id: The WHOOP User who performed the sleep activity.
        created_at: The time the sleep activity was recorded in WHOOP.
        updated_at: The time the sleep activity was last updated in WHOOP.
        start: Start time bound of the sleep. Represented as ISO 8601 timestamp.
        end: End time bound of the sleep. Represented as ISO 8601 timestamp.
        timezone_offset: The user's timezone offset at the time the sleep was recorded.
        nap: If true, this sleep activity was a nap for the user.
        score_state: The scoring state of the sleep activity. (Enum values: SCORED, PENDING_SCORE, UNSCORABLE)
        score_stage_summary_total_in_bed_time_milli: Total time in bed in milliseconds.
        score_stage_summary_total_awake_time_milli: Total awake time in milliseconds.
        score_stage_summary_total_no_data_time_milli: Total time with no data in milliseconds.
        score_stage_summary_total_light_sleep_time_milli: Total light sleep time in milliseconds.
        score_stage_summary_total_slow_wave_sleep_time_milli: Total slow-wave sleep time in milliseconds.
        score_stage_summary_total_rem_sleep_time_milli: Total REM sleep time in milliseconds.
        score_stage_summary_sleep_cycle_count: Total number of sleep cycles.
        score_stage_summary_disturbance_count: Total number of disturbances during sleep.
        score_sleep_needed_baseline_milli: Baseline sleep needed in milliseconds.
        score_sleep_needed_need_from_sleep_debt_milli: Additional sleep needed due to sleep debt, in milliseconds.
        score_sleep_needed_need_from_recent_strain_milli: Additional sleep needed due to recent strain, in milliseconds.
        score_sleep_needed_need_from_recent_nap_milli: Sleep needed adjustment due to recent nap, in milliseconds.
        score_respiratory_rate: Rate of breathing while asleep.
        score_sleep_performance_percentage: How well the Whoop User slept.
        score_sleep_consistency_percentage: How consistent the Whoop User slept.
        score_sleep_efficiency_percentage: How efficient the Whoop User slept.
        ####
        
        Example below between '####'
        
        ####
        <user>: What was my total baseline sleep needed, from 8/28/23 to 9/2/23, in hours?
        <assistant>: df[["score_sleep_needed_baseline_milli"]
        <assistant>: You're total baseline sleep needed was n 
        ####
        """
    ),
    Tool.from_function(
        func = today_date,
        name = "today_date",
        description=f"""
        Useful for when you need to know today's date when querying Whoop Sleep Data. Use to filter 'df' to a relevant subset. 
        'start' and 'end' date values in 'df' are of format ISO 8601 and look like this for example, '2023-08-29T13:26:21.600Z'. You will need to convert
        to match user query format with 'df' format.
        
        Examples of logic between '####'
        
        ####
        If today is 9/22/2023 then yesterday was 9/21/2023. 
        If today is 9/22/2023 then 2 (two) days ago was 9/20/2023, 3 days ago was 9/19/2023. So on and so forth through the calendar.
        ####
        """
    )
]

# change the value of the prefix argument in the initialize_agent function. This will overwrite the default prompt template of the zero shot agent type
agent_kwargs = {'prefix': f'You are friendly fitness assistant. You are tasked to assist the current user on questions related to their personal Whoop data. You have access to the following tools:'}
# {'prefix': f'You are friendly fitness assistant. You are tasked to assist the current user on questions related to their personal Whoop data. Always answer in a python dictionary where the key is the column from which you are referencing for the answer and the value of the dictionary is the numerical answer to the user query.'}

# initialize the LLM agent
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True, 
                         agent_kwargs=agent_kwargs
                         )
# define q and a function for frontend
def get_response(user_input):
    response = agent.run(user_input)
    return response

In [95]:
user_input = "How has my last week of sleep compared to all the time before that?"
get_response(user_input)



> Entering new AgentExecutor chain...
To compare the last week of sleep to all the time before that, I need to calculate the average sleep metrics for both periods and compare them.
Action: Whoop Sleep Data
Action Input: df
Observation:             id  user_id                created_at                updated_at  \
0    781257341  7036784  2023-09-22T10:39:00.978Z  2023-09-22T13:47:19.278Z   
1    780520172  7036784  2023-09-21T13:40:26.533Z  2023-09-21T13:40:31.454Z   
2    779342236  7036784  2023-09-20T11:21:19.182Z  2023-09-20T13:40:40.853Z   
3    778465925  7036784  2023-09-19T13:05:21.938Z  2023-09-19T13:40:38.089Z   
4    777277162  7036784  2023-09-18T10:46:29.871Z  2023-09-18T13:45:55.665Z   
..         ...      ...                       ...                       ...   
195  574997277  7036784  2022-12-30T14:47:25.061Z  2022-12-30T15:00:58.786Z   
196  574360085  7036784  2022-12-29T14:30:44.786Z  2022-12-29T14:43:24.212Z   
197  573751581  7036784  2022-12-28T15:01:25.425Z 

<string>:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
<string>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


I have calculated the average sleep metrics for the last week. Now I need to calculate the average sleep metrics for all the time before that.

Action: Whoop Sleep Data
Action Input: df[df['start'] < '2023-09-15'].mean()

Observation: id                                                      6.640756e+08
user_id                                                 7.036784e+06
nap                                                     4.166667e-02
score_stage_summary_total_in_bed_time_milli             2.816850e+07
score_stage_summary_total_awake_time_milli              4.162903e+06
score_stage_summary_total_no_data_time_milli            1.622255e+03
score_stage_summary_total_light_sleep_time_milli        1.389344e+07
score_stage_summary_total_slow_wave_sleep_time_milli    4.678478e+06
score_stage_summary_total_rem_sleep_time_milli          5.432063e+06
score_stage_summary_sleep_cycle_count                   5.182292e+00
score_stage_summary_disturbance_count                   1.298958e+01
score_

<string>:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
<string>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4304 tokens. Please reduce the length of the messages.

## manual eval

In [54]:
# user_input = "On what day was my longest time in bed and what was that time in bed?"
max_value = df_2['score_stage_summary_total_in_bed_time_milli'].max()
corresponding_date = df_2[df_2['score_stage_summary_total_in_bed_time_milli'] == max_value]['start'].iloc[0]
print(max_value)
print(corresponding_date)

# CORRECT

40282645
2023-02-12T03:33:46.355Z


## problem queries and notes

- what was my sleep 10 days ago?
- llm should provide more context with answers, like colloquially what column they used to get answer

## Tool gives python string that looks like dict then convert to json dict

In [44]:
import re

str_output = get_response(user_input)
json_like_part = re.search(r'\{.*\}', str_output).group(0)
json_str = json_like_part.replace("'", '"')
json_output = json.loads(json_str)
print(json_output)



> Entering new AgentExecutor chain...
To calculate the average time in bed, I need to find the mean of the 'score_stage_summary_total_in_bed_time_milli' column in the 'df' dataframe.
Action: Whoop Sleep Data
Action Input: df['score_stage_summary_total_in_bed_time_milli'].mean()
Observation: 28249793.58
Thought:The average time in bed is 28249793.58 milliseconds.
Final Answer: {'score_stage_summary_total_in_bed_time_milli': 28249793.58}

> Finished chain.
{'score_stage_summary_total_in_bed_time_milli': 28249793.58}


# dev set for testing - see Ng Eval 2

In [ ]:
msg_ideal_pairs_set = [
    
    # eg 0 - standard query
    {'customer_msg':"""What was the sleep id for date 9/22/2023?""",
     'ideal_answer':{
        'id':set(X)
        }
    },

    # eg 1 - sum/difference/avg w dates/criterion
    {'customer_msg':"""What was the sum of all my time slept between 9/22/2023 and 9/21/2023?""",
     'ideal_answer':{
        'score_stage_summary_total_in_bed_time_milli':set(X)
        }
    },
    # eg 2 - max/min w dates/criterion like dependent on other conditions
    {'customer_msg':f"""What was max time slept between 9/22/2023 and 9/21/2023?""",
     'ideal_answer':{
        'score_stage_summary_total_in_bed_time_milli':set(X)
        }
    }

'''
    # eg 3 - stacked: longest time in bed (max/min) and what was that time (standard query)
    {'customer_msg':f"""When was my longest time in bed and what was that time?""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['SmartX ProPhone']),
        'Cameras and Camcorders':set(
            ['FotoSnap DSLR Camera']),
        'Televisions and Home Theater Systems':set(
            ['CineView 4K TV', 'SoundMax Home Theater','CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV'])
        }
    }, 
    
    # eg 4 - stacked: highest time in bed and what was that time
    {'customer_msg':"""tell me about the CineView TV, the 8K one, Gamesphere console, the X one.
I'm on a budget, what computers do you have?""",
     'ideal_answer':{
        'Televisions and Home Theater Systems':set(
            ['CineView 8K TV']),
        'Gaming Consoles and Accessories':set(
            ['GameSphere X']),
        'Computers and Laptops':set(
            ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'])
        }
    }
'''
]
